In [1]:
import pandas as pd
import numpy as np
from sklearn import tree

In [2]:
#botdata = pd.read_csv('botAccounts.csv')
#gooddata= pd.read_csv('GoodAccounts.csv', encoding = 'ISO-8859-1')

In [2]:
testbot = pd.read_csv("/Users/lilixu/Desktop/ML/Machine-Learning-Project/data/bots_data.csv", encoding = 'ISO-8859-1')
testgood = pd.read_csv("/Users/lilixu/Desktop/ML/Machine-Learning-Project/data/nonbots_data.csv", encoding = 'ISO-8859-1')

In [3]:
df = testbot.append(testgood)

In [146]:
testbot.shape

(1056, 20)

In [147]:
testgood.shape

(1176, 20)

In [143]:
df.shape

(2232, 20)

In [46]:
#modify the column name to fit
def adjust_column_name(df):
    for x in df.columns.values:
        df = df.rename(columns={x: x.lower()})
    df = df.rename(columns={'listed_count' : 'listedcount'})

In [6]:
# split the dataset to training & testing
from sklearn.cross_validation import train_test_split
train, test = train_test_split(df, test_size = 0.2)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
df = testbot.append(testgood).fillna("")

In [161]:
def nb_dt(nb_column_name):
    train, test = train_test_split(df, test_size = 0.2)
    numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count','nb_result']
    count_vect = CountVectorizer()
    
    def convert(column):
        X_word_counts = count_vect.fit_transform(column)
        count_vect.vocabulary_.get(u'algorithm')
        tf_transformer = TfidfTransformer(use_idf=False).fit(X_word_counts)
        X_word_tf = tf_transformer.transform(X_word_counts)
        return X_word_tf.todense()
    
    def convert2(column):
        X_word_counts = count_vect.transform(test[nb_column_name])
        tf_transformer = TfidfTransformer(use_idf=False).fit(X_word_counts)
        X_word_tf = tf_transformer.transform(X_word_counts)
        return X_word_tf.todense()
    
    #Train
    clf = tree.DecisionTreeClassifier(random_state=0)
    M_clf = MultinomialNB().fit(convert(train[nb_column_name]), train['bot'])
    train['nb_result'] = M_clf.predict(convert(train[nb_column_name]))
    clf.fit(train[numfeatures],train["bot"])
    #Test
    test['nb_result'] = M_clf.predict(convert2(train[nb_column_name]))
    
    #Evaluate
    precison = precision_score(clf.predict(test[numfeatures]),test['bot'])
    recall = recall_score(clf.predict(test[numfeatures]),test['bot'])
    f1 = f1_score(clf.predict(test[numfeatures]),test['bot'])
    roc = roc_auc_score(clf.predict(test[numfeatures]),test['bot'])
    return [precison, recall, f1, roc]

In [184]:
for _ in range(20):
    print(nb_dt('description'))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0.83027522935779818, 0.82648401826484019, 0.82837528604118993, 0.83210165825522708]
[0.84375, 0.87096774193548387, 0.8571428571428571, 0.85939691444600286]
[0.84057971014492749, 0.82464454976303314, 0.83253588516746402, 0.84240702064422845]
[0.88118811881188119, 0.84761904761904761, 0.86407766990291268, 0.87317661241711886]
[0.87980769230769229, 0.84331797235023043, 0.86117647058823532, 0.86731116008815867]
[0.89861751152073732, 0.8666666666666667, 0.88235294117647067, 0.88378378378378386]
[0.86818181818181817, 0.86818181818181817, 0.86818181818181828, 0.87021425710853029]
[0.83743842364532017, 0.82524271844660191, 0.83129584352078234, 0.8441566289328446]
[0.89177489177489178, 0.88034188034188032, 0.88602150537634405, 0.88148549416155042]
[0.8303571428571429, 0.85321100917431192, 0.84162895927601822, 0.84363607227274551]
[0.87692307692307692, 0.79534883720930227, 0.8341463414634146, 0.84595028067361666]
[0.85781990521327012, 0.86602870813397126, 0.86190476190476195, 0.8699891439829521

In [158]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [159]:
def dt():
    df = testbot.append(testgood)
    numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count']
    train, test = train_test_split(df, test_size = 0.2)
    clf.fit(train[numfeatures],train["bot"])
    precison = precision_score(clf.predict(test[numfeatures]),test['bot'])
    recall = recall_score(clf.predict(test[numfeatures]),test['bot'])
    f1 = f1_score(clf.predict(test[numfeatures]),test['bot'])
    roc = roc_auc_score(clf.predict(test[numfeatures]),test['bot'])
    return [precison, recall, f1, roc]

In [185]:
for _ in range(20):
    print(dt())

[0.7981220657276995, 0.8133971291866029, 0.80568720379146919, 0.81636243013951992]
[0.82938388625592419, 0.87064676616915426, 0.84951456310679618, 0.86215265137725994]
[0.77880184331797231, 0.85353535353535348, 0.81445783132530114, 0.83038213459900201]
[0.85148514851485146, 0.81516587677725116, 0.83292978208232449, 0.84402361635472722]
[0.81132075471698117, 0.83091787439613529, 0.82100238663484482, 0.83212560386473433]
[0.83105022831050224, 0.84651162790697676, 0.83870967741935476, 0.84351443464314335]
[0.78280542986425339, 0.83574879227053145, 0.80841121495327106, 0.81787439613526591]
[0.78971962616822433, 0.82843137254901966, 0.80861244019138756, 0.82162309368191733]
[0.84070796460176989, 0.85201793721973096, 0.84632516703786187, 0.84565182575272269]
[0.84848484848484851, 0.76712328767123283, 0.80575539568345311, 0.81777217015140591]
[0.82211538461538458, 0.75330396475770922, 0.78620689655172415, 0.79256107328794545]
[0.86046511627906974, 0.87264150943396224, 0.86651053864168626, 0.8